# Manifold Learning and Dimensionality Reduction: Linear approaches

Starting off, let's take a moment to consider the unique challenges we often encounter in many biomedical applications. You'll often find yourself dealing with datasets that come with a high number of features, but not so many examples. It's a bit like trying to tell a detailed story with only a few words!

Now, this unique situation could increase the chances of overfitting. Why so? Well, when we go into higher dimensions, it's like spreading the same amount of data thinner and thinner, just like butter on an ever-growing slice of bread. This results in a feature space that's filled more and more sparsely.

Let's picture it this way: if we divide a region of space into regular cells, similar to a beehive structure, the number of these cells grows exponentially as the dimensionality of the space increases. Just imagine the honeycomb getting bigger and bigger, but with the same amount of honey to fill it! In the world of data, we're faced with a similar challenge: we need to efficiently manage our feature space to prevent overfitting and make the most out of our data.

<img src="imgs/curseofdimensionality.png" style="max-width:100%; width: 50%; max-width: none">

Now, to fill these cubes, we need a lot of data. Without that, we're more likely to stumble upon a separating hyperplane by chance, making our model more prone to overfitting. So, the challenge here is to trim down our feature space, so it doesn't overshadow the number of data examples. And we have a couple of awesome tools for this - feature selection, which you'll learn about later, and manifold learning, which we'll delve into right now.

Manifold Learning is a set of techniques that help us plot the data into a lower-dimensional space, exploiting the data's intrinsic geometry. Most real-world data obeys physical laws and has limited degrees of freedom, which means that they often live on a low-dimensional 'manifold', even when they're in a high-dimensional space. For instance, take a look at this 'Swiss roll model':

<img src="imgs/swissrollmodel.png" style="max-width:100%; width: 75%; max-width: none">

While it seems to be in three dimensions, its geometry is actually best represented in two dimensions (imagine unrolling it). Manifold learning techniques help to map data to a new space that reveals the underlying geometry more clearly, allowing simpler learning methods to cluster or classify the data.

There are various manifold learning methods, divided into linear (transforming data into a subspace) and non-linear methods (leveraging the local neighbourhood structure of the data). In this lecture, we'll discuss the linear approaches, specifically PCA and ICA. And in the upcoming session, we'll explore Laplacian Eigenmaps and Spectral clustering for learning more complex, non-linear structures.

Now, let's get into the specifics! Here are the generic modules that we'll be using throughout this notebook:



---


In many biomedical applications we will be faced with datasets that are represented with high dimensional featuresets but relatively few examples. This increases the chances of overfitting as, in ever higher dimensions, the same amount of data fills the featurespace more and more sparsely. Specifically, if we divide a region of space into regular cells we can see that the number of such cells grows exponentially with the dimensionality of the space:

<img src="imgs/curseofdimensionality.png" style="max-width:100%; width: 50%; max-width: none">

The problem with an exponentially large number of cells is that we would need an exponentially large quantity of training data in order to ensure that the cells are not empty. Without this the chance of finding a separating hyperplane by chance becomes easier, and we become prone to overfitting.  What we need therefore is some way of reducing the featurespace such that the number of features does not significanlty exceed that of the number of data examples. This can be done in a number of ways: feature selection (which you will learn about in future lectures) and manifold learning, which we describe here.

Manifold Learning describes a family of methods that seek to map the data into a lower dimensional space by exploiting the underlying geometry of the data.  Typically this is made possible because real world data are subject to physical laws and thus have limited degrees of freedom. This tends to mean that data generated by real world processes live on a low dimensional manifold, embedded within the high dimensional feature space. To explain what this means we can use a fairly artificial example: the 'Swiss roll model:'

<img src="imgs/swissrollmodel.png" style="max-width:100%; width: 75%; max-width: none">

Here it is clear that whilst the data lives in three dimensions, the geometry of the data is better represented in two (the result of unrolling the embedded representation). This underlying geometry can be determined by exploiting the local neighbourhood structure of the data. In this way it can be shown that the euclidean distance between two different points on the roll in three dimensions can vastly mispresent the true distance along the underlying manifold.

Thus, manifold learning techniques are designed to map the data to a new space that better represents the underlying geometry of the data, in this way enabling the use of simpler learning methods that can more sensitivitely cluster or classify the underlying data. Many different methods for manifold learning exist and these can be split into linear methods (that seek to find a sub-space of the data through linearly transforming the data) and non-linear approaches that more successully utlise the local neighbourhood structure of the data. In this lecture we will discuss linear approaches: specifically PCA and ICA. Next week will will introduce Laplacian Eigenmaps and Spectral clustering, which will allow learning of much more complex non-linear structures.

Importing generic modules used throughout the notebook:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, Image
from sklearn.decomposition import PCA

%matplotlib inline

## Linear Transforms

Let's kick things off by chatting about linear transformations. They play a starring role in all the methods we're going to explore in this lecture. They're kind of like the bread and butter of our learning today - simple yet fundamental. So, buckle up and get ready to step into the amazing world of linear transformations!

A linear transformation, $ T:\mathbb{R}^n \rightarrow \mathbb{R}^m$ , is a function which satisfies

1. $T(\mathbf{x_1} +\mathbf{x_2})=T(\mathbf{x_1})+ T(\mathbf{x_2}) \; \forall \mathbf{x_1}, \mathbf{x_2} \in \mathbb{R}^n $  (additivity)
2. $T(\alpha \mathbf{x_1})=\alpha T(\mathbf{x_1}) \; \forall \mathbf{x_1} \in \mathbb{R}^n ; \alpha \in  \mathbb{R} $ (scalar multiplicativy)
3. Preserves origin


The action of a matrix on a vector space is a linear transform. When you hear about a matrix acting on a vector space, think of it as performing a linear transformation. It's a bit like a director (the matrix) guiding an actor (the vector space) to perform a role (linear transformation). It's one of those fundamental aspects of linear algebra that's essential to so many fields, from computer graphics to machine learning.

i.e. consider the action of the matrix A=$\begin{pmatrix}
3 & 0 \\ 0 & 1 \\ \end{pmatrix} $ on the vector $\begin{pmatrix} x \\ y \\ \end{pmatrix} $

$ \begin{pmatrix} x' \\  y'\\ \end{pmatrix}=\begin{pmatrix} 3 & 0 \\ 0 & 1 \\ \end{pmatrix} \begin{pmatrix} x \\ y\\ \end{pmatrix} = x \begin{pmatrix} 3 \\ 0\\ \end{pmatrix} + y \begin{pmatrix} 0 \\ 1\\ \end{pmatrix}$

Applied to a cluster of grid points we can clearly see this transform as a stretch in the x-axis. Just imagine you've got a cluster of grid points, like a bunch of stars in the night sky. Now, when we apply a linear transformation (represented by our matrix), something pretty cool happens. The cluster of points gets stretched along the x-axis!

Think of it like pulling on a piece of elastic - the points spread out, but they still maintain their relative positions to each other. It's a simple, yet powerful visualisation of how linear transformations work.

<img src="imgs/gridtransform.png" style="max-width:100%; width: 75%; max-width: none">

This may be more clearly shown through the following matplotlib animation developed using code adapted from https://dododas.github.io/linear-algebra-with-python/posts/16-12-29-2d-transformations.html
and https://notgnoshi.github.io/linear-transformations/

Note: you need ffmpeg installed for this to run. In Anaconda Prompt run

```conda install -c menpo ffmpeg```


In [ ]:
import animate_transforms as an
from matplotlib import  rc

rc('animation', html='html5')


A = np.column_stack([[3, 0], [0, 1]])
anim = an.animate_transform(A, repeat=True)
anim

**Optional Exercise:**
How about a little optional challenge? Let's experiment with different matrix forms and see the various kinds of linear transformations we can pull off. Think of it like trying out different dance moves with our matrix and vector space!

For instance, give these matrices a whirl:

- A = $\begin{pmatrix} 2 & 0.5 \\ 0.5 & 1 \\ \end{pmatrix} $
- A = $\begin{pmatrix} 1 & 2 \\ 0 & 1 \\ \end{pmatrix} $
- and a rotation of the form  A=$\begin{pmatrix} cos\theta & -sin\theta \\ sin\theta & cos\theta \\ \end{pmatrix} $

Each matrix will produce a different transformation, kind of like how different recipes result in different cakes.

So, go ahead, play around and have some fun! You're not just learning about linear transformations here, you're also honing your problem-solving skills. And don't forget to think about what type of transformations these matrices are describing! It's like a detective game with mathematics. Happy investigating!

### Eigendecomposition

Think of Eigendecomposition as a magician's trick that simplifies complex linear operations, presenting them in a clearer, diagonalized format. It's a bit like taking a twisted piece of wire and straightening it out into a simpler, easier-to-handle straight line. The stars of this trick are eigenvectors. These are special vectors in linear transforms that have a unique ability: they stay the same, except for a scale factor, even after the transform is applied. It's like they have an invisibility cloak against transformations! As if they have a special immunity against transformations!

To get a bit more technical (with a helping hand from e.g [wikipedia](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors), Eigendecomposition presents linear operators in a simplied, diagonalised form. Specifically, eigenvectors of linear transforms are (non-zero) vectors which remain unchanged (with exception of a scale factor) following application of the transform.


<img src="imgs/mona_lisa_example.png" style="max-width:100%; width: 50%; max-width: none">

When a shear transformation (imagine it like a big push) is applied to our special eigenvectors (like our blue vector here), they're like the immovable rock in the face of a mighty river. They stay completely unchanged in their direction. On the contrary, a vector that isn't an eigenvector (like our red vector here) doesn't hold its ground. Its direction gets altered when the same shear transformation is applied.

And what about how much our eigenvectors stretch or shrink under the transformation? That's where eigenvalues step into the spotlight. You can think of eigenvalues as the precise measurement that shows by how much the length of an eigenvector changes in the transformation. In other words, they signify the scaling factor that changes the magnitude of the eigenvector during the transformation.

Thus eigenvectors $\mathbf{v}$ of a matrix $\mathbf{A}$ obey the following equation:

$$A\mathbf{v}=\lambda \mathbf{v}$$

and thus may be found by starting from the characteristic equation:

$$A\mathbf{v}-\lambda \mathbf{v}=0$$
$$ \rightarrow (A-\lambda \mathbf{I})\mathbf{v}=0$$
$$ \rightarrow |A-\lambda \mathbf{I}|=0$$

Since we have already defined $\mathbf{v}$ to be non zero. Here $\mathbf{I}$ is the identity matrix. Note, the determinant ($ \det \mathbf{B}$ or $|B|$) of a 2D matrix:

$$ \begin{pmatrix}
a & b \\
d & c \\
\end{pmatrix} $$

is defined as:

$$ |B|= ac-bd $$

Thus, for linear transform:

$$ A= \begin{pmatrix}
3 & 0 \\
0 & 1 \\
\end{pmatrix} $$

We estimate $|A-\lambda \mathbf{I}|= 0 $ as:

$$ \rightarrow \begin{pmatrix}
3-\lambda & 0 \\
0 & 1-\lambda \\
\end{pmatrix}=0 $$

$$ \rightarrow (3-\lambda)(1-\lambda)=0 $$.

Thus, $(3-\lambda)$ and $(1-\lambda)$ must equal zero, and the eigenvalues of this linear transform are 3 and 1. Accordingly estimating the eigenvector for $\lambda=3$ results from:

<img src="imgs/eigenvectors_forlambda_3.png" style="max-width:100%; width: 30%; max-width: none">

Representing a stretch in the x axis. And for $\lambda=1$:

<img src="imgs/eigenvectors_forlambda_1.png" style="max-width:100%; width: 30%; max-width: none">

Representing the unmodified y axis

### A note on Affine Transforms (optional)

Here's a little optional nugget of knowledge about Affine Transforms, especially for those of you who might have come across them in the field of image registration.

So, what's an Affine Transformation? It's a function that's linear in nature, but with a bit of a twist. Unlike a strict linear transform, an Affine Transformation bends the rules of additivity a bit; for example, Let's consider a function that looks like this:
$f(\mathbf{x})=b\mathbf{x} +c $. This, my friends, is an Affine Transformation. Now, let's put it to the additivity test, which states that T(x1 + x2) should be the same as T(x1) + T(x2) for all vectors x1 and x2.

$$T(\mathbf{x_1} +\mathbf{x_2})=T(\mathbf{x_1})+ T(\mathbf{x_2}) \; \forall \mathbf{x_1}, \mathbf{x_2} \in \mathbb{R}^n $$  

On the right hand side, we get

$RHS=b \mathbf{x_1} + c + b \mathbf{x_2} +c =b(\mathbf{x_1} +\mathbf{x_2}) +2c$

On the left hand side, we get

$LHS=b(\mathbf{x_1} +\mathbf{x_2}) +c$

$LHS \neq RHS$

As you can see, the left hand side isn't equal to the right hand side, which means our function doesn't strictly meet the conditions of a linear transform.

So what's going on here? Well, an Affine Transformation is actually a combination of a linear transformation and a translation (that's the + c part). It's a little like a linear transform that's gone off on a scenic detour!

There you have it - a sneak peek into the slightly rebellious world of Affine Transforms!

## Exercise 1: Estimating eigenvectors and values from a linear transformation matrix

Estimate (on paper) the eigenvectors and values of:

$$ A= \begin{pmatrix}
2 & 1 \\
1 & 2 \\
\end{pmatrix} $$

Thre solution is given in the lecture slides.

# Principal Component Analysis (PCA)

So, remember when we were discussing eigendecomposition of linear transforms? That cool trick that lets us discover a new basis? Well, it turns out this can help us transform our data in a neat and efficient way.

In the context of PCA (Principal Component Analysis), we're specifically hunting for a new basis that reflects the variance of our data in the best possible way. Think of it as finding the best angle to view a sculpture so you can see the most detail.

We're aiming to project our data onto this new basis. This is like rotating our perspective to this optimal angle. When we do this, the covariance of our data becomes diagonalised - it simplifies into a form that's much easier to understand and work with.

Let's bring this to life with a concrete example. Do you recall the dataset we explored in week 2, where we were looking at neonatal gestational age in relation to brain volume? Well, we're going to revisit that data and see how these concepts apply. Exciting stuff!

By way of example, let's return to a dataset you first saw previously in week 2: neonatal gestational age versus brain volume.

<img src="imgs/ga_versus_brain_volume.png" style="max-width:100%; width: 50%; max-width: none">

So, we've got a pretty strong link between these two features – the neonatal gestational age and the brain volume. This correlation is so robust that, even when we account for some noise, it wouldn't be a stretch to say that we could roughly guess one from the other. Kind of like knowing that if it's raining, there are probably clouds in the sky!

In other words, it seems like all the important info, all the relationships that matter in this data set, might be best captured through a single projection, which we'll call $\mathbf{u_1}$.

It's like condensing a whole novel into a single, poignant sentence that captures the essence of the story. This way, we're cutting out all the extra noise and focusing on the main event. So, let's continue our journey and see where this single projection takes us!

Take a look at this! It's like these two features - neonatal gestational age and brain volume - are best buddies. They're so strongly connected that, even with a bit of noise thrown into the mix, you could almost predict one if you knew the other.
So, what does this mean for our data? Well, all the important relationships, the ones that really tell the story, could potentially be captured using just a single projection, which we're calling $\mathbf{u_1}$. Think of it like getting the 'gist' of a whole movie from watching just one really awesome scene. Isn't data science fun?

<img src="imgs/ga_versus_brain_volume_with_comp.png" style="max-width:100%; width: 50%; max-width: none">

As we progress further on our journey into PCA, we'll have to tackle the task of estimating this new basis $\mathbf{u}$, such that when we project our data points onto this sub-space, the variance of these projected points is maximized. Now, I know it sounds a bit tricky, but bear with me!

To make things a little clearer, let's imagine a simpler dataset with just four points. In our visualisation, we'll mark the original points in cheery orange, represent the new basis with a bold red line, and show the points projected onto this line in lively green:


<img src="imgs/simplePCA.png" style="max-width:100%; width: 40%; max-width: none">

Here's a fun fact: The projection of one vector ($\mathbf{x}$) onto another ($\mathbf{u_1}$) can be calculated using a simple equation. Take a look:

<img src="imgs/vectorprojection.png" style="max-width:100%; width: 40%; max-width: none">

This equation actually stems from the dot product, a concept you might remember from your earlier math adventures:

$$ \mathbf{x.u_1} = |\mathbf{x}||\mathbf{u_1}|cos \theta $$

So, the projection $|\mathbf{x}|cos \theta = \frac{\mathbf{x.u_1}}{ |\mathbf{u_1}|}$ or $\mathbf{x.u_1}$ if $\mathbf{u_1}$ is a unit vector.

Alright, let's make an assumption here. Let's say $\mathbf{u_1}$ **is** a unit vector. After we've projected our data, the variance of the projected data would be given by this neat formula:

$$\frac{1}{n-1} \sum_{n=1}^N  (\mathbf{u_1^Tx_n}-\mathbf{u_1^T\bar{x}})^2 = \mathbf{u_1^TSu_1} $$

where, $ S= \frac{1}{n-1} \sum_{n=1}^N  (\mathbf{x_n}-\mathbf{\bar{x}})^T (\mathbf{x_n}-\mathbf{\bar{x}}) $ and $\mathbf{\bar{x}}$ is the mean of the data.

Imagine you're looking for the perfect photo angle to capture the beauty of a landscape. You'd probably move around a bit until you found that "sweet spot" where everything just seems to fall into place, right? That's exactly what we're trying to do with $\mathbf{u_1}$. We want to adjust it so that when we project our data onto it, it gives us the clearest picture of the data's variance.

This is where the magic of PCA happens. We find this ideal projection by maximizing the projected variance $\mathbf{u_1^TSu_1}$ with respect to $\mathbf{u_1}$, but we also need to set some boundaries. We can't let $\mathbf{u_1}$ go off into infinity, so we make sure it stays as a unit vector: $\mathbf{u_1}^T\mathbf{u_1}=1 $. We use a tool called a Lagrange multiplier $\lambda_1$ to enforce this:

We seek $\mathbf{u_1}$ such that when we project our data onto it, we will maximise the representation of the variance. Thus the derivation of PCA can be found by maximising the projected variance $\mathbf{u_1^TSu_1}$  w.r.t. $\mathbf{u_1}$, whilst constraining any solution to prevent $\mathbf{u_1} \rightarrow \infty$ and instead return a unit vector: $\mathbf{u_1}^T\mathbf{u_1}=1 $. This can be enforced through use of a Lagrange multiplier $\lambda_1$:

$$ \mathbf{u_1^TSu_1} + \lambda_1(1-\mathbf{u_1}^T\mathbf{u_1}) $$

The way to maximize this is to take its derivative with respect to $\mathbf{u_1}$:

$$  \mathbf{Su_1} - \lambda_1\mathbf{u_1}=0 $$
$$ \rightarrow  \mathbf{Su_1}=\lambda_1\mathbf{u_1} $$

Now, what if we want to take this idea further, into higher dimensions? We need to make sure our basis vectors $\mathbf{u_i}$ don't interfere with each other - they need to be orthogonal, uncorrelated:

$$ cov(\mathbf{u_1},\mathbf{u_2})=\mathbf{u_2^TSu_1}= \mathbf{u_2}\lambda_1 \mathbf{u_1} $$

With this additional constraint in mind, our formula becomes a little more complex, but we can maximize it the same way, by taking the derivative with respect to $\mathbf{u_2}$:

$$ \mathbf{u_2^TSu_2} + \lambda_2(1-\mathbf{u_2}^T\mathbf{u_2}) - \phi\mathbf{u_2}^T\mathbf{u_1} $$

$$  \mathbf{Su_2}-\lambda_2\mathbf{u_2} - \phi\mathbf{u_1} = 0 $$

Doing some math with $\mathbf{u_1}$. Left multiplying through by $\mathbf{u_1}$ gives:
$$  \mathbf{u_1^TSu_2}-\lambda_2\mathbf{u_1^Tu_2} - \phi\mathbf{u_1^Tu_1} = 0 $$

We find that due to our orthogonality condition, $\mathbf{u_1^TSu_2}$ and $\lambda_2\mathbf{u_1^Tu_2}$ must both be zero. Therefore, $\phi\mathbf{u_1^Tu_1}$ must also be zero:

$$  \mathbf{u_1^TSu_2}-\lambda_2\mathbf{u_1^Tu_2} - \phi\mathbf{u_1^Tu_1} = 0 $$
$$ \rightarrow 0 - 0 - \phi\mathbf{u_1^Tu_1} = 0$$

Alright, now that we've jumped through those mathematical hoops, let's take a step back and appreciate what we've uncovered. All those zeros we derived? They show us that $ \mathbf{Su_2}=\lambda_2\mathbf{u_2} $. Simply put, our second basis vector, $\mathbf{u_2}$, is the second eigenvector of our data's covariance matrix. That's quite the revelation!

### A note on Langrangian multipliers (optional but fun!)

The above derivation takes a Langrangian approach to constrained optimisation.  This supports estimation the local maxima and minima of a function subject to equality constraints without need for explicit parameterisation of the constraints.

The method we used here is called Lagrangian optimization, and it's a fantastic tool for optimizing functions when you have certain constraints. It's like trying to find the best path up a mountain while making sure you don't stray off a designated trail - sometimes the direct path isn't the best one!

Let's focus on our trusty tools: functions and constraints. In our situation, we want to maximize a function $f(x)$ while ensuring that the constraint $g(x)=0$ holds true.

In Lagrangian terms, we represent this as the Lagrangian function $L(x,\lambda)=f(x) -\lambda g(x)$. This function cleverly incorporates both our original function and our constraint, neatly packaged with the Lagrange multiplier $\lambda$.

In this case, our "mountain" was the function $ \mathbf{u_1Su_1^T} $ and our "trail" was the constraint $\mathbf{u_1}^T\mathbf{u_1}=1 $. By using Lagrange multipliers, we're able to find the peak of our function without breaking our constraint rule. And this isn't just handy for PCA – we use this method in a bunch of other techniques, like support vector machines, too!

## Estimating PCA eigenvectors from the Covariance matrix

Alright, let's take a look at our next piece of the PCA puzzle: the eigen-decomposition of our mean-centered data covariance matrix $\mathbf{S}=\frac{1}{n-1} \mathbf{(X- \mathbf{\bar X})(X-\mathbf{\bar X})^T}$.

Thus, all principal eigenvectors may be obtained from eigen-decomposition of the mean-centred data covariance matrix $\mathbf{S}=\frac{1}{n-1} \mathbf{(X- \mathbf{\bar X})(X-\mathbf{\bar X})^T}$, where $\mathbf{X} \in \mathbb{R}^{m\times n}$, n= number of samples/examples and m =numbers of features, and mean $\mathbf{\bar X} =\frac{1}{n} \sum_i^n \mathbf{ X_i} $;

This results in the following eigenvalue equation (in matrix representation):

$\mathbf{SU}=\mathbf{\Lambda U}$,


with $\mathbf{U} \in \mathbb{R}^{m \times m}$ and $\mathbf{\Lambda} \in \mathbb{R}^{m \times m}$; $\mathbf{\Lambda}$ is a diagonal matrix e.g.

$\mathbf{\Lambda}= \begin{pmatrix}
\lambda_1 & 0 & 0 & \dots & 0 \\
0 & \lambda_2 & 0 & \dots & 0 \\
\vdots & \vdots  &  \vdots  & \vdots  & \vdots  \\
0 & 0 & 0 & \dots & \lambda_m \\
\end{pmatrix}$

With the PCA basis estimated as eigenvectors $\mathbf{U}$, all data $\mathbf{X}$ may be projected into this new basis as $\mathbf{Y} = \mathbf{U^TX}$


## Exercise 2: Estimating PCA bases from data covariance matrices

Given the data matrix found in the file random_data.txt, estimate the PCA eigenspace from the eigenvectors of the data covariance matrix, and use this to project all data into the new basis.

In the next window complete the function for estimation of pca from the sample covariance matrix ($\mathbf{S}=\frac{1}{n-1} \mathbf{X X^T}$). Replace ```None``` values with correct code so as to :

1.  mean centre the data ($\mathbf{X}=\mathbf{X}-\mathbf{\bar X}$)
2.  estimate the data covariance matrix
3.  Evaluate the eigenvectors and values of the covariance matrix.

Hints:
 - you may use numpy's mean function (```np.mean(..)```), but check that the dimensionality of your mean vector matches what you expect. Your goal is to estimate the mean and variance of the features.
 - Please implement the sample covariance; you can check its accuracy against the output of ```np.cov()```
 - For eigenvalue and eigenvectors use numpys linalg.eig(..) function (see Numpy.ipynb and https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html for more details)

In [ ]:
# create function to estimate PCA from covariance matrix
def pca_from_covariance(X):

    """
    Function to estimate PCA from sample covariance matrix:

    Input args:
        X : data matrix with shape (n_features, n_samples)

    Output args:
        u : eigenvectors of the sample covariance matrix
        d : eigenvalues of the sample covariance matrix
        X : centered data
    """

    # 2.1.1 estimate mean vector
    X_mean=None

    # 2.1.2 mean center
    X=None

    # 2.2 estimate data covariance matrix
    S=None

    # 2.3 estimate eigenvalues and vectors using inbuilt numpy functions
    d,u=None


    return u,d,X

4. Now apply your function to the data loaded from random_data.txt, project all data onto the PCA eigenspace

We also plot the results of the transformation:

In [ ]:
# load data
X=np.loadtxt('random_data.txt')
print(X.shape)

# perform pca using pca_from_covariance(X)
vecs,vals,X_centered=pca_from_covariance(X)
print('Eigenvalues are:', vals)
print("Eigenvectors are: \n {}".format(vecs))

# 2.4 project data
Y=None
print("Projected coordinates for first two data points are: \n {}".format( Y[:,:3]))

# plot original data
plt.figure()
plt.plot(X[0],X[1],'o')
plt.title('Data in native coordinates')
plt.xlabel('$X_1$')
plt.ylabel('$Y_1$')

# plot transformed data
plt.figure()
plt.plot(Y[0],Y[1],'x')
plt.title('Data in PC coordinates')
plt.xlabel('$PC 1$')
plt.ylabel('$PC 2$')


We can now visualise the principal components in relation to our original data. For this we can plot them as vectors, using the eigenvalue to define the  magnitude of the vector:

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)


In [ ]:
# plot original data
plt.figure()
plt.scatter(X[0],X[1], alpha=0.2)
plt.title('Data in native coordinates')
plt.xlabel('$X_1$')
plt.ylabel('$Y_1$')

for length, vector in zip(vals, vecs.transpose()):
    v = vector * 3 * np.sqrt(length)
    draw_vector(np.mean(X,axis=1),np.mean(X,axis=1) + v)
plt.axis('equal');


## Singular Value Decomposition (SVD)

Alternatively PCA eigenvectors and values may be estimated through Singular Value Decomposition (SVD). This decomposition allows spectral (eigen) decomposition of any matrix (including non-square matrices).

For any $(m \times n)$ matrix $\mathbf{X}$, where $n \geq m $ SVD is defined as:

$$ \mathbf{X}=\mathbf{U\Sigma V^T}$$

Where, $\mathbf{U} \in \mathbb{R}^{m\times m }$, with orthonormal columns ($\mathbf{U}^T\mathbf{U} = \mathbf{I}$), represents the left singular vectors of matrix $\mathbf{X}$; $\mathbf{V} \in \mathbb{R}^{n\times n }$, ($\mathbf{V}^T\mathbf{V} = \mathbf{I})$ represents the right singular vectors of matrix $\mathbf{X}$; and $\mathbf{\Sigma} \in \mathbb{R}^{m\times n }$ is a diagonal matrix with positive or zero elements, called the singular values.

The relationship to PCA may be seen as follows: first construct two square matrices  $\mathbf{XX^T}$ and $\mathbf{X^TX}$:

$$\rightarrow  \mathbf{XX^T}=\mathbf{U\Sigma V^TV\Sigma^T U^T= U \Sigma \Sigma^T U^T}$$
$$\rightarrow  \mathbf{X^TX}= \mathbf{V \Sigma^T \Sigma  V^T}$$

For the case that $n \geq m $ we can show that $\mathbf{XX^T}$ and $\mathbf{X^TX}$ share $m$ eigenvalues, and the remaining $n-m$ are zero. This can be seen from the eigendecomposition of $\mathbf{XX^T= U \Sigma \Sigma^T U^T}$, which (as a square matrix of size $m \times m $) has $m$ eigenvalues and eigenvectors. These correspond to the columns of $\mathbf{U}$ and the squared diagonal elements of $\mathbf{\Sigma}$, as shown by multiplying both sides by $\mathbf{U}$ :

$$  \mathbf{XX^T= U \Sigma \Sigma^T U^T}$$
$$\rightarrow  \mathbf{XX^TU}= \mathbf{U \Sigma \Sigma^T }$$

Thus for a single eigenvector $\mathbf{u}$  and corresponding singular value $\sigma^2$ :

$$  \mathbf{XX^Tu}= \mathbf{\sigma^2 u}$$

Multiply both sides with $\mathbf{X^T}$:

$$  \mathbf{X^TXX^Tu}= \mathbf{\sigma^2 X^Tu}$$
$$ \rightarrow \mathbf{(X^TX)X^Tu}= \mathbf{\sigma^2 X^Tu}$$


Demonstrates that $\mathbf{v=X^Tu}$ is an eigenvector of $\mathbf{X^TX}$, also with eigenvalue $\sigma^2$. Thus $\mathbf{XX^T}$ and $\mathbf{X^TX}$ share $m$ eigenvalues. All remaining $n-m$ eignvalues are zero, as shown by the fact that the matrix of singular values has rectangular form:

<img src="imgs/sigma.png" style="max-width:100%; width: 40%; max-width: none">
<br>

Note, the singular values are related to the eigenvalues of the covariance matrix ($\lambda$, defined above) as:

$$\lambda_i=\frac{\sigma_i^2}{n-1}$$



### Why use SVD instead of the covariance method?

For rank deficient matrices (where $𝑛 \gg 𝑚 $, or vice versa) SVD is much more numerically stable, since many $\lambda_𝑖$ will be very near 0; the covariance matrix will be rank-deficient and ill-conditioned; therefore eigen-decomposition will become numerically unstable


## Exercise 3

Given the data matrix found in the file random_data.txt, estimate the PCA eigenspace through Singular Value Decomposition (SVD).  Project all data into the new basis (as before).

1. First complete the function ```pca_from_svd``` replacing ```None``` with the correct code.
2. In the following window apply the function to the matrix $\mathbf{X}$ as before (done for you).
3. Estimate the explained variance (equivalent to the eigenvalues of the covariance matrix) by transforming the singular values
4. Project the data into the new basis and plot the result
5. Finally compare against sklearn pca method

Hints:
- Use https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html


In [ ]:
# create function to estimate PCA from SVD
def pca_from_svd(X):

    """
    Function to estimate PCA through SVD

    Input args:
        X : data matrix with shape (n_features, n_samples)

    Output args:
        u : left singular vectors
        d : singular values
        v : right singular vectors
        X : centered data
    """
    # 3.1.1  mean center
    X=None

    # 3.1.2 estimate eigenvalues and vectors using SVD
    u,d,v=None

    return u,d,v,X

In [ ]:
# 3.2 perform pca using pca_from_svd(X)
leftvecs,vals,rightvecs,X=pca_from_svd(X)

# 3.3. convert singular values to explained variance
lambdas=None

print('SVD singular values are:', vals)
print('Explained Variance  :  {}'.format(lambdas))
print("left singular vectors are: \n {}".format(leftvecs))

# plot original data with SVD eigenvectors (why will the vectors not show up!!)
plt.scatter(X[0],X[1], alpha=0.2)
plt.title('Data in native coordinates')
plt.xlabel('$X_1$')
plt.ylabel('$Y_1$')

for length, vector in zip(vals, leftvecs.transpose()):
    v = vector * 3 * np.sqrt(length)
    draw_vector(np.mean(X,axis=1),np.mean(X,axis=1) + v)
plt.axis('equal');

# 3.4 transform data
Y=None

plt.figure()
plt.plot(Y[0],Y[1],'x')



Now compare to the Scikit-Learn implementation: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn.decomposition import PCA

#3.5.1 instantiate and fit model (two lines)
model=None
mode. # complete this line

# get attributes: 1) explained variance (lambdas); 2) singular values (gammas); 3) components
explained_variance=model.explained_variance_
singular_values=model.singular_values_
components=model.components_

print('Explained variance (lambdas): {}'.format(explained_variance))
print('Singular values (gammas):  {}'.format(singular_values))
print('Principal components: \n {}'.format(components))



## PCA for dimensionality reduction

Returning to the case made at the beginning of the lecture, in many cases we may wish to reduce the dimensionality of our featurespace. This may be down to the curse of dimensionality, such that the problem has many more features than examples, but equally you may find, that even for a relatively small number of features, many are linearly correlated, redundant for the problem at hand, or may simply reflect noise in the data.

In these cases PCA may be used for dimensionality reduction. This occurs naturally through SVD since, for $ n \gt m$, $n-m$ singular values will be zero. Nevertheless, further reductions can be achieved by selecting the eigenvectors corresponding to the $k$ largest eigenvalues ($k \lt m$).

The choice of $k$ eigenvectors may be made by examining the cumulative distribution of eigenvalues, and selecting a cut-off at the point where growth levels off.

<img src="imgs/cdf_eigenvalues_cutoff.png" style="max-width:100%; width: 40%; max-width: none">


## Exercise 4: Dimensionality Reduction

Apply PCA based dimensionality reduction to the problem of prediction of neonatal gestation age from regional volumes. This builds from examples used in regression lectures.

1.  Run through notebook: 5.2.PCA_GA_estimation.ipynb
